In [ ]:
class CancerTreatmentSimulator:
    def __init__(self):
        # Initialize the board with empty spaces
        self.board = [' '] * 9

    # Print the current state of the board
    def print_board(self):
        print('-------------')
        for i in range(0, 9, 3):
            print(f'| {self.board[i]} | {self.board[i+1]} | {self.board[i+2]} |')
            print('-------------')

    # Update the board with the player's move
    def update_board(self, move, player):
        if self.board[move] == ' ':
            self.board[move] = player
            return True
        else:
            return False

    # Check for a winner or tie
    def check_winner(self):
        # Check rows, columns, and diagonals for a win
        for i in range(0, 9, 3):
            if self.board[i] == self.board[i+1] == self.board[i+2] != ' ':
                return self.board[i]
        for i in range(3):
            if self.board[i] == self.board[i+3] == self.board[i+6] != ' ':
                return self.board[i]
            
        #diagonals
        if self.board[0] == self.board[4] == self.board[8] != ' ':
            return self.board[0]
        if self.board[2] == self.board[4] == self.board[6] != ' ':
            return self.board[2]
        
        # Check for tie
        if ' ' not in self.board:
            return 'tie'

        return None


class AIPlayer:
    def __init__(self, player):
        self.player = player

    # Minimax algorithm for AI decision making
    def minimax(self, simulator, depth, is_maximizing):
        # Check if there is a winner or if the game ended in a tie
        result = simulator.check_winner()

        if result is not None:
            # If the AI wins, return a score 1 /favoring quicker victories
            if result == self.player:
                return 1 #10 - depth
            # If it's a tie, return 0
            elif result == 'tie':
                return 0
            # If the opponent wins, return a -1 score -1 /favoring quicker defeats
            else:
                return -1 #depth - 10

        if is_maximizing:
            # If it's the AI's turn to maximize its score
            best_score = float('-inf')
            # Iterate over all possible moves
            for i in range(9):
                # If the space is empty
                if simulator.board[i] == ' ':
                    # Place the AI's move on the board
                    simulator.board[i] = self.player
                    # Recursively call minimax for the opponent's turn
                    score = self.minimax(simulator, depth + 1, False)
                    # Undo the move for backtracking
                    simulator.board[i] = ' '
                    # Update the best score
                    best_score = max(score, best_score)
            return best_score
        else:
            # If it's the opponent's turn to minimize the AI's score
            best_score = float('inf')
            # Iterate over all possible moves
            for i in range(9):
                # If the space is empty
                if simulator.board[i] == ' ':
                    # Place the opponent's move on the board
                    simulator.board[i] = 'O' if self.player == 'X' else 'X'
                    # Recursively call minimax for the AI's turn
                    score = self.minimax(simulator, depth + 1, True)
                    # Undo the move for backtracking
                    simulator.board[i] = ' '
                    # Update the best score
                    best_score = min(score, best_score)
            return best_score

    # Get the best move for the AI player using minimax algorithm
    def get_best_move(self, simulator):
        # Initialize variables for tracking the best move and its score
        best_score = float('-inf')
        best_move = None

        # Iterate over all possible moves
        for i in range(9):
            # If the space is available
            if simulator.board[i] == ' ':
                # Place the AI's move on the board
                simulator.board[i] = self.player
                # Call minimax to calculate the score for this move
                score = self.minimax(simulator, 0, False)
                # Undo the move for backtracking
                simulator.board[i] = ' '
                # Update the best move and its score if necessary
                if score > best_score:
                    best_score = score
                    best_move = i

        return best_move


def start_simulator():
    # Give information about game to user
    print("Welcome to the Cancer Treatment Decision Simulator!")
    print("You will play as the doctor (X) against the AI opponent (O).")
    print("Let's start the game!")

    simulator = CancerTreatmentSimulator()
    ai_player = AIPlayer('O')
    
    # Ask user if they want to move first
    user_moves_first = input("Do you want to move first? (yes/no): ").lower() == 'yes'

    # Set current player based on user's choice
    current_player = 'X' if user_moves_first else 'O'

    while True:
        simulator.print_board()

        if current_player == 'X':
            # User's turn
            move = int(input("Enter your move (1-9): "))
            if move < 1 or move > 9 or simulator.board[move - 1] != ' ':
                print("Invalid move! Please try again.")
                continue
            else:
                simulator.update_board(move - 1, current_player)
        else:
            # AI's turn
            move = ai_player.get_best_move(simulator)
            print(f"The AI moves to cell {move + 1}")
            simulator.update_board(move, current_player)

        winner = simulator.check_winner()
        if winner:
            # Game over
            simulator.print_board()
            if winner == 'X':
                print("Congratulations! You successfully treated the patient!")
            elif winner == 'O':
                print("The cancer has overcome the treatment. You lost.")
            else:
                print("The treatment resulted in a tie.")
            break

        # Switch player for the next turn
        if current_player == 'X':
            current_player = 'O'
        else:
            current_player = 'X'

# Start the simulator
start_simulator()